# **Recommender Systems with Graph Neural Networks on MovieLens**

## Setting up the environment

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.1 MB/s eta 0:00:00


## Loading and preprocessing the MovieLens-Latest-Small dataset

### Loading the dataset using Pandas

In [3]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

### Encode Users and Items

In [4]:
import numpy as np

ratings = ratings[ratings['userId'].notna()]

user_id_mapping = {id: idx for idx, id in enumerate(ratings['userId'].unique())}
item_id_mapping = {id: idx for idx, id in enumerate(ratings['movieId'].unique())}

ratings.loc[:, 'userId'] = ratings['userId'].map(user_id_mapping)
ratings.loc[:, 'movieId'] = ratings['movieId'].map(item_id_mapping)

## Building the user-item interaction graph

###  Build the bipartite graph

In [99]:
import torch
from torch_geometric.data import Data

# 1. First, add device configuration at the start
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

num_users = ratings['userId'].nunique()
num_items = ratings['movieId'].nunique()
num_nodes = num_users + num_items

user_nodes = ratings['userId'].to_numpy()
item_nodes = ratings['movieId'].to_numpy() + num_users

edge_index = np.vstack((user_nodes, item_nodes))
edge_attr = torch.tensor(ratings['rating'].to_numpy(), dtype=torch.float)

x = torch.eye(num_nodes)
data = Data(x=x, edge_index=torch.tensor(edge_index, dtype=torch.long), edge_attr=edge_attr)

data = data.to(device)


Using device: cuda


## Implementing the LightGCN model with attention

In [100]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Step 1: Implementing LightGCN Layer with Attention
class LightGCNConv(MessagePassing):
    def __init__(self, **kwargs):
        super(LightGCNConv, self).__init__(aggr='add', **kwargs)

    def forward(self, x, edge_index):
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

# Step 2: Incorporating Attention Mechanisms
class LightGCNConvWithAttention(MessagePassing):
    def __init__(self, in_channels):
        super(LightGCNConvWithAttention, self).__init__(aggr='add')
        self.att = torch.nn.Parameter(torch.Tensor(1, in_channels * 2))
        torch.nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        x_cat = torch.cat([x_i, x_j], dim=-1)
        alpha = F.leaky_relu((x_cat * self.att).sum(dim=-1))
        alpha = F.softmax(alpha, dim=0)
        return alpha.unsqueeze(-1) * x_j

# Step 3: Defining the Complete Model
class LightGCNWithAttention(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim=256, num_layers=4, dropout=0.2):
        super().__init__()
        self.num_layers = num_layers
        self.embedding = torch.nn.Embedding(num_nodes, embedding_dim)
        self.layer_norm = torch.nn.LayerNorm(embedding_dim)
        self.dropout = torch.nn.Dropout(dropout)
        torch.nn.init.xavier_uniform_(self.embedding.weight)

        self.convs = torch.nn.ModuleList([
            LightGCNConvWithAttention(embedding_dim) for _ in range(num_layers)
        ])

    def forward(self, edge_index):
        x = self.embedding.weight
        all_embeddings = [x]

        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.layer_norm(x)
            x = self.dropout(x)
            all_embeddings.append(x)

        return torch.stack(all_embeddings, dim=0).mean(dim=0)

    def get_embedding(self, edge_index):
        embeddings = self.forward(edge_index)
        return embeddings

## Training the model

### Preparing Training and Test Sets

In [101]:
from sklearn.model_selection import train_test_split

# Split the data
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Create edge indices for training
train_edge_index = np.vstack((
    train_data['userId'].to_numpy(),
    train_data['movieId'].to_numpy() + num_users
))
train_edge_index = torch.tensor(train_edge_index, dtype=torch.long)

test_edge_index = np.vstack((
    test_data['userId'].to_numpy(),
    test_data['movieId'].to_numpy() + num_users
))

test_edge_index = torch.tensor(test_edge_index, dtype=torch.long)


### Defining Loss Function and Optimizer

In [134]:
model = LightGCNWithAttention(
    num_nodes=num_nodes,
    embedding_dim=256,
    num_layers=4,
    dropout=0.2
)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-5
)

### Training Loop

In [135]:
# 2. Move model to device
model = model.to(device)

# 3. Update training function with device handling
def train(model, optimizer):
    model.train()
    optimizer.zero_grad()

    # Get embeddings
    embeddings = model.get_embedding(data.edge_index.to(device))

    # Sample negative items
    batch_size = train_data['userId'].shape[0]
    neg_items = torch.randint(0, num_items, (batch_size,), device=device)

    # Get embeddings for users, positive and negative items
    user_emb = embeddings[train_data['userId'].to_numpy()].to(device)
    pos_item_emb = embeddings[train_data['movieId'].to_numpy() + num_users].to(device)
    neg_item_emb = embeddings[neg_items + num_users].to(device)

    # BPR loss
    pos_scores = (user_emb * pos_item_emb).sum(dim=1)
    neg_scores = (user_emb * neg_item_emb).sum(dim=1)
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

    loss.backward()
    optimizer.step()
    return loss.item()

## Evaluating the model

### RMSE on Test Set

In [136]:
from sklearn.metrics import root_mean_squared_error

def test(model):
    model.eval()
    with torch.no_grad():
        embeddings = model.get_embedding(data.edge_index.to(device))
        user_emb = embeddings[test_data['userId'].to_numpy()].to(device)
        item_emb = embeddings[test_data['movieId'].to_numpy() + num_users].to(device)
        preds = (user_emb * item_emb).sum(dim=1)
        rmse = root_mean_squared_error(test_data['rating'], preds.cpu().numpy())
    return rmse


### Recall@K

In [137]:
def recall_at_k(model, edge_index, k=10):
    model.eval()
    with torch.no_grad():
        # Get embeddings
        embeddings = model.get_embedding(edge_index.to(device))

        # Separate user and item embeddings
        user_emb = embeddings[:num_users]
        item_emb = embeddings[num_users:num_users+num_items]

        # Calculate scores for all user-item pairs
        scores = torch.mm(user_emb, item_emb.t())

        # Get top k items for each user
        _, top_k_items = scores.topk(k=k, dim=1)

        # Move tensors to CPU for numpy operations
        top_k_items = top_k_items.cpu()
        edge_index = edge_index.cpu()

        # Calculate recall
        hits = torch.zeros(num_users)
        for i in range(num_users):
            relevant_items = set((edge_index[1][edge_index[0] == i] - num_users).numpy())
            recommended_items = set(top_k_items[i].numpy())
            hits[i] = len(relevant_items & recommended_items) / len(relevant_items) if len(relevant_items) > 0 else 0

        return hits.mean().item()


In [138]:
def precision_at_k(model, edge_index, k=10):
    model.eval()
    with torch.no_grad():
        # Get embeddings
        embeddings = model.get_embedding(edge_index.to(device))

        # Separate user and item embeddings
        user_emb = embeddings[:num_users]
        item_emb = embeddings[num_users:num_users+num_items]

        # Calculate scores for all user-item pairs
        scores = torch.mm(user_emb, item_emb.t())

        # Get top k items for each user
        _, top_k_items = scores.topk(k=k, dim=1)

        # Move tensors to CPU for numpy operations
        top_k_items = top_k_items.cpu()
        edge_index = edge_index.cpu()

        # Calculate precision
        precision_scores = torch.zeros(num_users)
        for i in range(num_users):
            relevant_items = set((edge_index[1][edge_index[0] == i] - num_users).numpy())
            recommended_items = set(top_k_items[i].numpy())
            hits = len(relevant_items & recommended_items)
            precision_scores[i] = hits / k  # Precision is hits divided by k

        return precision_scores.mean().item()

## Putting It All Together

### Training and Evaluation Loop

In [139]:
def train_and_test(model, optimizer, scheduler, save_file):
    best_recall = 0
    patience = 10
    patience_counter = 0
    num_epochs = 200

    for epoch in range(num_epochs):
        loss = train(model, optimizer)
        rmse = test(model)
        recall = recall_at_k(model, test_edge_index, k=10)
        precision = precision_at_k(model, test_edge_index, k=10)

        print(f'Epoch {epoch+1}, Loss: {loss:.4f}, RMSE: {rmse:.4f}, Recall@10: {recall:.4f}, Precision@10: {precision:.4f}')

        scheduler.step(loss)

        if recall > best_recall:
            best_recall = recall
            torch.save(model.state_dict(), save_file)
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    # Load best model
    model.load_state_dict(torch.load(save_file))

train_and_test(model, optimizer, scheduler, 'best_model.pt')

Epoch 1, Loss: 0.6931, RMSE: 3.6511, Recall@10: 0.0117, Precision@10: 0.0280
Epoch 2, Loss: 0.6931, RMSE: 3.6506, Recall@10: 0.0489, Precision@10: 0.0887
Epoch 3, Loss: 0.6931, RMSE: 3.6497, Recall@10: 0.0543, Precision@10: 0.0926
Epoch 4, Loss: 0.6930, RMSE: 3.6483, Recall@10: 0.0544, Precision@10: 0.0915
Epoch 5, Loss: 0.6928, RMSE: 3.6466, Recall@10: 0.0541, Precision@10: 0.0902
Epoch 6, Loss: 0.6926, RMSE: 3.6443, Recall@10: 0.0542, Precision@10: 0.0898
Epoch 7, Loss: 0.6923, RMSE: 3.6415, Recall@10: 0.0538, Precision@10: 0.0898
Epoch 8, Loss: 0.6919, RMSE: 3.6383, Recall@10: 0.0527, Precision@10: 0.0885
Epoch 9, Loss: 0.6915, RMSE: 3.6344, Recall@10: 0.0525, Precision@10: 0.0885
Epoch 10, Loss: 0.6909, RMSE: 3.6300, Recall@10: 0.0525, Precision@10: 0.0885
Epoch 11, Loss: 0.6903, RMSE: 3.6250, Recall@10: 0.0523, Precision@10: 0.0884
Epoch 12, Loss: 0.6896, RMSE: 3.6194, Recall@10: 0.0526, Precision@10: 0.0887
Epoch 13, Loss: 0.6888, RMSE: 3.6131, Recall@10: 0.0531, Precision@10: 0.

<ipython-input-139-42c8ed27003e>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_file))


## Baseline Model

In [140]:
# Updated train function
def train(model, optimizer):
    model.train()
    optimizer.zero_grad()

    # Get embeddings
    embeddings = model.get_embedding(data.edge_index.to(device))
    print('test')

    pos_rank, neg_rank = model(data.edge_index, train_edge_index).chunk(2)
    loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=train_edge_index.unique(),
    )

    loss.backward()
    optimizer.step()
    return loss.item()


In [141]:
def train_and_test(model, optimizer, scheduler, save_file):
    best_recall = 0
    patience = 10
    patience_counter = 0
    num_epochs = 200

    for epoch in range(num_epochs):
        loss = train(model, optimizer)
        rmse = test(model)
        recall = recall_at_k(model, test_edge_index, k=10)
        precision = precision_at_k(model, test_edge_index, k=10)

        print(f'Epoch {epoch+1}, Loss: {loss:.4f}, RMSE: {rmse:.4f}, Recall@10: {recall:.4f}, Precision@10: {precision:.4f}')

        scheduler.step(loss)

        if recall > best_recall:
            best_recall = recall
            torch.save(model.state_dict(), save_file)
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    # Load best model
    model.load_state_dict(torch.load(save_file))

In [142]:
from torch_geometric.nn import LightGCN
# Initialize the baseline LightGCN model
baseline_model = LightGCN(num_nodes=num_nodes, embedding_dim=256, num_layers=4).to(device)

# Define optimizer
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-5
)

In [143]:
train_and_test(baseline_model, optimizer, scheduler, 'best_model_baseline.pt')

test
Epoch 1, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0005, Precision@10: 0.0039
test
Epoch 2, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0009, Precision@10: 0.0041
test
Epoch 3, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0004, Precision@10: 0.0038
test
Epoch 4, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0005, Precision@10: 0.0038
test
Epoch 5, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0005, Precision@10: 0.0036
test
Epoch 6, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0006, Precision@10: 0.0038
test
Epoch 7, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0007, Precision@10: 0.0038
test
Epoch 8, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0008, Precision@10: 0.0036
test
Epoch 9, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0008, Precision@10: 0.0033
test
Epoch 10, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0008, Precision@10: 0.0031
test
Epoch 11, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0011, Precision@10: 0.0036
test
Epoch 12, Loss: 0.6931, RMSE: 3.6514, Recall@10: 0.0014, Precision@10: 0.0041
test
Epoch 13

<ipython-input-141-416316b66e6f>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_file))
